In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd

In [20]:
# We read data and specified Tokenizer
tokenizer = Tokenizer()
data = pd.read_csv("/Users/maheshbabu/Desktop/NLP-Next-Word-Predictor/ArticlesJan2017.csv")

corpus = data['snippet'].str.lower().tolist()

# corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)

total_words = len(tokenizer.word_index) + 1 


print(tokenizer.word_index)
print(total_words)

{'the': 1, 'a': 2, 'to': 3, 'of': 4, 'and': 5, 'in': 6, 'for': 7, 'on': 8, 'is': 9, 'that': 10, 'with': 11, 'be': 12, 'as': 13, 'but': 14, 'are': 15, 'it': 16, 'president': 17, 'an': 18, 'his': 19, 'by': 20, 'new': 21, 'what': 22, 'you': 23, 'at': 24, 'about': 25, 'trump': 26, 'has': 27, 'have': 28, 'from': 29, 'this': 30, 'was': 31, 'who': 32, 'can': 33, 'not': 34, 'will': 35, 'or': 36, 'their': 37, 'how': 38, 'they': 39, 'more': 40, 'donald': 41, 'do': 42, '—': 43, 'trump’s': 44, 'he': 45, 'when': 46, 'times': 47, 'our': 48, 'one': 49, 'most': 50, 'its': 51, 'people': 52, 'would': 53, 'now': 54, 'your': 55, 'mr': 56, 'it’s': 57, 'york': 58, 'may': 59, 'could': 60, 'us': 61, 'some': 62, 'been': 63, '”': 64, 'i': 65, 'we': 66, 'like': 67, 'j': 68, 'up': 69, 'over': 70, 'news': 71, 'than': 72, 'no': 73, 'american': 74, 'women': 75, 'so': 76, 'even': 77, 'should': 78, 'country': 79, 'years': 80, 'care': 81, 'which': 82, 'two': 83, 'her': 84, 'them': 85, 'all': 86, 'were': 87, 'why': 88, 

In [3]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = np.array(token_list[:i+1])
        input_sequences.append(n_gram_sequence)

In [4]:
max_sequence_len = max([len(x) for x in input_sequences])

In [5]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [6]:
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [7]:
glove_dictionary = {}
with open('/Users/maheshbabu/Desktop/NLP-Next-Word-Predictor/glove.6B.100d.txt', encoding="utf8") as file:
    for each_line in file:
        words_in_line, coeff_cients = each_line.split(maxsplit=1)
        coeff_cients = np.array(coeff_cients.split(),dtype = float)
        glove_dictionary[words_in_line] = coeff_cients

In [8]:
embedding_matrix = np.zeros((total_words, 100))

for key, value in tokenizer.word_index.items():
    if key in glove_dictionary.keys():
        embedding_matrix[value, :] = glove_dictionary[key]
    else:
        embedding_matrix[value, :] = np.zeros(100)

In [9]:
from tensorflow.keras import backend as K

def perplexity(y_true, y_pred):
    cross_entropy = K.categorical_crossentropy(y_true, y_pred)
    perplexity = K.exp(K.mean(cross_entropy))
    return perplexity

In [10]:
# model = tf.keras.Sequential()

# embed_layer = tf.keras.layers.Embedding(input_dim = total_words, output_dim = 100,
#                                    input_length = max_sequence_len - 1, weights = [embedding_matrix])

# model.add(embed_layer)

# model.add(Bidirectional(LSTM(150)))

# model.add(Dense(128, activation='relu'))

# model.add(Dense(total_words, activation='softmax'))

# adam = Adam(lr=0.01)

# model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=[perplexity])

# history = model.fit(xs, ys, epochs=30,validation_split=0.1, verbose=1)

In [12]:
model = tf.keras.Sequential()

embed_layer = tf.keras.layers.Embedding(input_dim = total_words, output_dim = 100,
                                        input_length = max_sequence_len - 1,
                                        weights = [embedding_matrix],
                                        trainable = False) # Freeze embedding weights

model.add(embed_layer)

model.add(Bidirectional(LSTM(256, return_sequences=True)))

model.add(Bidirectional(LSTM(128)))

model.add(Dense(256, activation='relu'))

model.add(tf.keras.layers.Dropout(0.5))

# Add dense layer with softmax activation
model.add(Dense(total_words, activation='softmax'))

# Compile model
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=[perplexity])

# Fit model
history = model.fit(xs, ys, epochs=30, validation_split=0.1, verbose=1)


Epoch 1/10


/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2023-04-09 20:49:09.252700: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-09 20:49:10.660107: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-09 20:49:10.923806: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-09 20:49:10.936081: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-09 20:49:11.225427: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-09 20:49:11.237089: I tensorflow/core/

432/432 [==============================] - ETA: 0s - loss: 7.2861 - perplexity: 1623.4368

2023-04-09 20:49:38.906061: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-09 20:49:39.004689: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-09 20:49:39.013458: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-09 20:49:39.162825: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-09 20:49:39.171478: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


432/432 [==============================] - 32s 65ms/step - loss: 7.2861 - perplexity: 1623.4368 - val_loss: 7.1163 - val_perplexity: 1297.5903
Epoch 2/10
432/432 [==============================] - 26s 61ms/step - loss: 6.9614 - perplexity: 1145.6968 - val_loss: 7.1217 - val_perplexity: 1314.4183
Epoch 3/10
432/432 [==============================] - 26s 61ms/step - loss: 6.8142 - perplexity: 984.6687 - val_loss: 7.1630 - val_perplexity: 1390.3256
Epoch 4/10
432/432 [==============================] - 27s 61ms/step - loss: 6.6775 - perplexity: 868.1196 - val_loss: 7.1834 - val_perplexity: 1434.2567
Epoch 5/10
432/432 [==============================] - 27s 63ms/step - loss: 6.5433 - perplexity: 766.6441 - val_loss: 7.2479 - val_perplexity: 1545.7861
Epoch 6/10
432/432 [==============================] - 27s 61ms/step - loss: 6.4213 - perplexity: 661.6061 - val_loss: 7.2756 - val_perplexity: 1598.2227
Epoch 7/10
432/432 [==============================] - 27s 62ms/step - loss: 6.2860 - perple

In [13]:
seed_text = "I don't think it should be a"
next_words = 10
#Prediction
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted = np.argmax(predicted, axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
    print(seed_text)

2023-04-09 20:53:59.642315: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-09 20:53:59.739211: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-09 20:53:59.750529: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-09 20:54:00.043857: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-09 20:54:00.052490: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


I don't think it should be a new
I don't think it should be a new york
I don't think it should be a new york york
I don't think it should be a new york york york
I don't think it should be a new york york york york
I don't think it should be a new york york york york york
I don't think it should be a new york york york york york york
I don't think it should be a new york york york york york york york
I don't think it should be a new york york york york york york york york
I don't think it should be a new york york york york york york york york york


In [14]:
predictions = model.predict(xs)

480/480 [==============================] - 13s 27ms/step


In [15]:
np.argmax(predictions,axis = 1)

array([17, 21,  4, ..., 12,  1, 21])

In [16]:
predictions.shape

(15348, 4779)

In [17]:
labels

array([  50,  296,    5, ..., 4777,   48, 4778], dtype=int32)

In [18]:
import tensorflow as tf
loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(ys, predictions))
perplexity = tf.exp(loss)
print('Perplexity: {:.2f}'.format(perplexity.numpy()))

Perplexity: 352.21
